In [ ]:
def czyszczenie_danych(df):
    # Usuwanie podwojonych wierszy
    df.drop_duplicates(inplace = True)
    #
    # Usuwanie części kolumn o charakterze identyfikacyjnym
    df.drop(df[['Unnamed: 0','ID_ZLA', 'ID_PL', 'ID_UB', 'ID_LEKARZ', 'ID_ZOZ', 'STAN_NA',
               'O_PAR01_STAN_NA']], axis = 1, inplace = True)
    # Określenie liczby kolumn z jedną wartością:
    liczba_kolumn_z_1_wartością = df.nunique()
    # Określenie kolumn do usunięcia
    do_usunięcia = [i for (i, v) in enumerate(liczba_kolumn_z_1_wartością) if v == 1]
    # Usunięcie kolumn z jedną wartością:
    df.drop(df.columns[do_usunięcia], axis = 1, inplace = True)
    #
    # Wstawianie brakujących wartości typu float
    imputer = SimpleImputer(strategy = 'mean')
    df1 = df.select_dtypes('float64')
    df_imp1  = pd.DataFrame(imputer.fit_transform(df1), columns = df1.columns)
    # Wstawianie brakujących wartości typu int64
    df2 = df.select_dtypes('int64')
    imputer = SimpleImputer(strategy ='most_frequent')
    df_imp2 = pd.DataFrame(imputer.fit_transform(df2), columns = df2.columns)
    # Wstawianie brakujących wartości typu object
    df3 = df.select_dtypes('object')
    imputer = SimpleImputer(strategy = 'constant', fill_value = 'NIEZNANE')
    df_imp3 = pd.DataFrame(imputer.fit_transform(df3), columns = df3.columns)
    df = pd.concat([df_imp1, df_imp2, df_imp3], axis = 1)
    # Usuwanie wartości odstających z kolumn typu float64
    lof = LocalOutlierFactor()
    mniam = lof.fit_predict(df_imp1)
    maska = mniam != -1
    df_pcz = df[maska]
    print("Postacie ramek danych w funkcji czyszczenie")
    print(df1.shape, df_imp1.shape)
    print(df2.shape, df_imp2.shape)
    print(df3.shape, df_imp3.shape)
    return df_pcz

In [ ]:
def czyszczenie_danych_ulepszone(df):
    """Czyszczenie ramki danych, występującej jako parametr formalny.

    Argumenty:
        df: obiekt pandas DataFrame, który ma być oczyszczony.

    Zwraca:
        oczyszczoną ramkę danych.
    """

    # 1. Usuwanie jednego z identycznych wierszy:
    df.drop_duplicates(inplace=True)

    # 2. Usuwanie niepotrzebnej kolumny z identyfikatorami oraz datami i czasem:
    #   Utworzenie listy kolumn do usunięcia i ich usuwanie
    kolumny_do_usunięcia = ['Unnamed: 0', 'ID_ZLA', 'ID_PL', 'ID_UB', 'ID_LEKARZ',
                         'ID_ZOZ', 'STAN_NA', 'O_PAR01_STAN_NA', 'DATA_WP']
    df.drop(columns=kolumny_do_usunięcia, errors='ignore', inplace=True)

    # 3. Usuwanie kolumn, zawierających tylko jedną wartość:
    df = df.loc[:, df.nunique() > 1]

    # 4. Wstawianie brakujących wartości przy użyciu modułu 'pipeline':
    #   - Użycie klasy sklearn.pipeline.Pipeline zapewnia lepszą organizację
    #   - Strategie wstawiania są różne w zależności od typu kolumny
    #
    # 'Pipeline' jest narzędziem, które pozwala na łączenie różnych kroków
    # przetwarzania danych i modelowania w jeden, spójny proces.
    # Pipeline umożliwia automatyzację i uporządkowanie kolejnych etapów przetwarzania
    # danych, takich jak przetwarzanie wstępne (np. skalowanie, kodowanie),
    # selekcja cech i właściwe modelowanie.

    # Główne zalety użycia Pipeline w scikit-learn to:

    # 1. Uproszczenie kodu: Pipeline pozwala na zdefiniowanie całego procesu
    # przetwarzania i modelowania w jednym miejscu, co czyni kod bardziej
    # czytelnym i łatwym do zarządzania.
    # 2. Unikanie błędów: Automatyzacja kolejnych kroków przetwarzania zmniejsza
    # ryzyko popełnienia błędów, takich jak pominięcie ważnego etapu.
    # Łatwe walidowanie krzyżowe: Pipeline może być łatwo użyty w walidacji krzyżowej,
    # co ułatwia ocenę modelu i pozwala uniknąć wycieku danych ("data leakage").

    from sklearn.pipeline import Pipeline


    # SimpleImputer jest klasą w module sklearn.impute biblioteki scikit-learn,
    # która służy do uzupełniania brakujących wartości (missing values) w zbiorze danych.
    # Uzupełnianie brakujących wartości jest ważnym krokiem w przygotowaniu danych
    # do analizy i modelowania, ponieważ wiele algorytmów uczenia maszynowego
    # nie radzi sobie z brakującymi danymi.

    # SimpleImputer oferuje kilka strategii uzupełniania brakujących wartości:
    # Średnia (mean): Uzupełnianie brakujących wartości średnią z kolumny.
    # Mediana (median): Uzupełnianie brakujących wartości medianą z kolumny.
    # Najczęstsza wartość (most_frequent): Uzupełnianie brakujących wartości najczęściej występującą wartością w kolumnie.
    # Stała wartość (constant): Uzupełnianie brakujących wartości określoną stałą wartością

    from sklearn.impute import SimpleImputer


    # ColumnTransformer jest klasą w module sklearn.compose biblioteki scikit-learn,
    # która pozwala na stosowanie różnych transformacji do różnych kolumn w zbiorze danych.
    # Jest to przydatne, gdy różne kolumny wymagają różnych rodzajów przetwarzania,
    # takich jak skalowanie, kodowanie kategorialnych zmiennych lub uzupełnianie
    # brakujących wartości.

    # ColumnTransformer umożliwia zdefiniowanie różnych transformacji dla różnych kolumn
    # i łączy je w jeden, spójny proces. Można określić, które kolumny mają być
    # przetwarzane przez które transformacje, co czyni ColumnTransformer
    # bardzo elastycznym narzędziem do przygotowania danych.

    from sklearn.compose import ColumnTransformer

    cechy_numeryczne = df.select_dtypes(include=['number']).columns
    cechy_kategorialne = df.select_dtypes(include=['object']).columns

    potok_numeryczny = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
    ])

    potok_kategorialny = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='NIEZNANE')),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', potok_numeryczny, cechy_numeryczne),
            ('cat', potok_kategorialny, cechy_kategorialne),
        ])

    df = pd.DataFrame(preprocessor.fit_transform(df),
                      columns=cechy_numeryczne.tolist() + cechy_kategorialne.tolist(),
                      index=df.index)  # Keep original index

    # 5. Usuwanie wartości odstających za pomocą LocalOutlierFactor:
    #   - Stosuje się tylko do cech numerycznych



    # LocalOutlierFactor (LOF) jest algorytmem wykrywania anomalii (outliers)
    # zawartym w module sklearn.neighbors biblioteki scikit-learn.
    # Algorytm LOF służy do identyfikowania punktów danych, które są "oddalone"
    # od reszty danych, co oznacza, że są rzadkie lub niezwykłe w porównaniu
    # z resztą zbioru danych.

    # LOF działa na zasadzie porównywania gęstości lokalnej punktów danych.
    # Punkty, które mają niższą gęstość lokalną w porównaniu z ich sąsiadami,
    # są uznawane za anomalie. Algorytm LOF jest szczególnie przydatny w przypadkach,
    # anomalie nie są jednoznacznie oddalone od reszty danych,
    # ale są rzadkie w lokalnym otoczeniu.

    # Główne parametry LocalOutlierFactor to:

    # n_neighbors: Liczba sąsiadów do rozważenia przy obliczaniu gęstości lokalnej.
    # contamination: Proporcja anomalii w zbiorze danych. Używana do ustalenia progu decyzyjnego.
    # novelty: Jeśli True, algorytm jest używany do wykrywania anomalii w nowych danych (novelty detection). Jeśli False, algorytm jest używany do wykrywania anomalii w danych treningowych.
    # Metoda fit_predict zwraca etykiety, gdzie -1 oznacza anomalie, a 1 oznacza normalne punkty.

    from sklearn.neighbors import LocalOutlierFactor

    lof = LocalOutlierFactor()
    outlier_mask = lof.fit_predict(df[cechy_numeryczne]) != -1
    df = df[outlier_mask]

    return df

In [ ]:
def przygotowanie_danych(df_prim):
    print("hihi")
    dane_wyj = czyszczenie_danych(df_prim)
    print("Postać danych dane_wyj")
    print(dane_wyj.shape)
    # Kodowanie danych typu 'int' oraz 'object'
    X, y = dane_wyj.drop(columns = 'fraud_wyk'), dane_wyj[['fraud_wyk']]
    print(type(X), X.shape, type(y), y.shape)
    Xfloat = X.select_dtypes('float64')
    Xkat = X.select_dtypes(exclude=['float64'])
    Xkat_dumm = pd.get_dummies(Xkat)
    X = pd.concat([Xfloat, Xkat_dumm], axis = 1)
#    dane_wyj = pd.concat([X, y], axis = 1)
    sc = StandardScaler()
    mmsc = MinMaxScaler()
    dane_wyj_float = dane_wyj.select_dtypes('float64')
#    dane_wyj_int = dane_wyj.select_dtypes('int64')
#    dane_wyj_obj = dane_wyj.select_dtypes('object')

    data_num = pd.concat([dane_wyj_float, dane_wyj['fraud_wyk']], axis = 1)
    print("Postać danych data_num")
    print(data_num.shape)
    selekcja_cech_num(data_num)

    data_kat = dane_wyj.select_dtypes(exclude=['float64'])
    print("Postać danych data_kat")
    print(data_kat.shape)


    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.33)

    print('W funkcji przygotowanie_danych', type(Xtrain), Xtrain.shape, type(ytrain), ytrain.shape,
          type(Xtest), Xtest.shape, type(ytest), ytest.shape)
    Xtrain_float = Xtrain.select_dtypes('float64')
    Xtest_float = Xtest.select_dtypes('float64')
    Xtrain_reszta = Xtrain.select_dtypes(exclude=['float64'])
    Xtest_reszta = Xtest.select_dtypes(exclude=['float64'])
    mmsc.fit(Xtrain_float)
    Xtrain_float = mmsc.transform(Xtrain_float)
    mmsc.transform(Xtest_float)
    Xtrain = pd.concat([Xtrain_float, Xtrain_reszta], axis = 1)
    Xtest = pd.concat([Xtest_float, Xtest_reszta], axis = 1)

    return Xtrain, Xtest, ytrain, ytest

In [ ]:
def przygotowanie_danych(df_prim):
    print("hihi")
    dane_wyj = czyszczenie_danych(df_prim)
    print("Postać danych dane_wyj")
    print(dane_wyj.shape)
    # Kodowanie danych typu 'int' oraz 'object'
    X, y = dane_wyj.drop(columns = 'fraud_wyk'), dane_wyj[['fraud_wyk']]
    print(type(X), X.shape, type(y), y.shape)
    Xfloat = X.select_dtypes('float64')
    Xkat = X.select_dtypes(exclude=['float64'])
    Xkat_dumm = pd.get_dummies(Xkat)
    X = pd.concat([Xfloat, Xkat_dumm], axis = 1)
#    dane_wyj = pd.concat([X, y], axis = 1)

    mmsc = MinMaxScaler()

    dane_wyj_float = dane_wyj.select_dtypes('float64')

    data_num = pd.concat([dane_wyj_float, dane_wyj['fraud_wyk']], axis = 1)
    print("Postać danych data_num")
    print(data_num.shape)

    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.33)

    print('W funkcji przygotowanie_danych', type(Xtrain), Xtrain.shape, type(ytrain), ytrain.shape,
          type(Xtest), Xtest.shape, type(ytest), ytest.shape)
    Xtrain_float = Xtrain.select_dtypes('float64')
    Xtest_float = Xtest.select_dtypes('float64')
    Xtrain_reszta = Xtrain.select_dtypes(exclude=['float64'])
    Xtest_reszta = Xtest.select_dtypes(exclude=['float64'])

    mmsc.fit(Xtrain_float)
    Xtrain_float = mmsc.transform(Xtrain_float)  # Przypisanie danych przekształconych
    Xtest_float = mmsc.transform(Xtest_float)   # Przypisanie danych przekształconych

    Xtrain = pd.concat([Xtrain_float, Xtrain_reszta], axis = 1)
    Xtest = pd.concat([Xtest_float, Xtest_reszta], axis = 1)

    return Xtrain, Xtest, ytrain, ytest